In [31]:
import requests
import re
import time
import json
from Ann_date_sql import *
#词云库
from wordcloud import WordCloud
import PIL .Image as image
import numpy as np
def get_comment(url):
    r = requests.get(url).content
    text=r.decode('gbk')
    #获取返回的json数据
    # print(text)
    data=text
    data1=data.replace("fetchJSON_comment98(", "");
    data2=data1.replace(");", "");
    # print(data2)
    data = json.loads(data2)#data1的内容为一个字典，用{}括起来的内容
    #字典清洗，提取comments内容
    for i in data['comments']:
        #商品名称
        productName = i['referenceName']
        #评论时间
        commentTime = i['creationTime']
        #评论内容
        content = i['content']
        #将数据插入数据库保存
        thing_insertData(productName,commentTime,content)
    print('ok')
    
#我爬取的时候总共0-35页每页10条数据 36页还有2条就不要了
#分批获取，防止封本人是0,11---11，21---21，35
for i in range(21,35):
    number=str(i)
    url = "https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=100011199522&score=1&sortType=6&page=%s&pageSize=10&isShadowSku=0&fold=1"%number
    get_comment(url)
    #防止封，加定时
    time.sleep(3)
print('爬取完成')
# 查询总共几条数据
a=thing_slectTable()
print(len(a))
# print(a)
#从数据库获取全部评论
b=thing_slectComment()
print(len(b))
# print(b)
#将数据写入txt文件
file = open('jingdongComments.txt','w')
for i in b:
    #暴力去掉无用字符写入txt文件
    strs=str(i)[2:][:-2].replace("\n", "")
    strs=strs.replace("&hellip;", "")
    strs=strs.replace("rdquo", "")
    strs=strs.replace("ldquo", "")
    strs=strs.replace("运行速度", "")
    strs=strs.replace("拍照效果", "")
    strs=strs.replace("待机时间", "")
    strs=strs.replace("外形外观", "")
    strs=strs.replace("其他特色", "")
    strs=strs.replace("屏幕音效", "")
    strs=strs.replace("n", "")
    file.write(strs)
file.close()
print('写入ok')
#读取txt文件
with open("jingdongComments.txt") as fp:
    text=fp.read()
    # 将文本放入WordCoud容器对象中并分析
    # 词云图片
    mask = np.array(image.open("2.jpg"))

    #字体：C:\Windows\Fonts\FZSTK.TTF  C:\Windows\Fonts\FZLTCXHJW.TTF  每个人的系统字符库里都有
    font="C:\Windows\Fonts\FZLTCXHJW.TTF"
    
    WordCloud =WordCloud(
        # 设置字体，不指定就会出现乱码
        font_path=font,  # 这个路径是pc中的字体路径
        
        # 设置背景色
        background_color='white',
        
        # 词云形状
        mask=mask,
        
        # 允许最大词汇
        max_words=100,
        
        # 最大号字体
        max_font_size=100,
        
        # 设置有多少种随机生成状态，即有多少种配色方案
        random_state=30,
        
        # 清晰度
        scale=3
    ).generate(text)
    
    image_produce = WordCloud.to_image()
    image_produce.show()
    
print('词云完成')

ok
完成
